In [1]:
import os
import dlt
import yaml
import requests
from openai import AzureOpenAI
from dataclasses import dataclass

@dataclass
class LLMServiceConfig:
    host: str
    api_key: str
    api_version: str
    model: str

with open("../OPENAI_API_KEY.yaml") as f:
    details = yaml.safe_load(f)["Glossary Terms Extraction Service"]
    
service_config = LLMServiceConfig(
    host=f"{details['protocol']}://{details['host']}",
    api_key=details['api_key'],
    api_version=details['api_version'],
    model=details['model']
)
client = AzureOpenAI(
    api_version=service_config.api_version,
    azure_endpoint=service_config.host,
    api_key=service_config.api_key
)
model = service_config.model
service_config.api_version

'2025-03-01-preview'

# Q1

In [2]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [3]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "This tool returns temperature for a city. If weather in city is not know, temperature is generated randomly from -5 to 35 degrees Celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city you want to know the temperature in"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [4]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

In [5]:
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'This tool returns temperature for a city. If weather in city is not know, temperature is generated randomly from -5 to 35 degrees Celsius.',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Name of the city you want to know the temperature in'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [6]:
developer_prompt = """
You're a weather forecasting assistant. 
You're given a question about weather in a specific city and you need to answer.

Use tools if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [7]:
chat.run()

You: stop


Chat ended.


# Q2

In [8]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [9]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "This tool set's temperature for a city and stores it permanently",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city you want to set the temperature for"
            },
            "temp": {
                "type": "number",
                "description": "Temperature you want to set"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [10]:
tools.add_tool(set_weather, set_weather_tool)

In [11]:
developer_prompt = """
You're a weather forecasting assistant. 
You're given a question about weather in a specific city and you need to answer.

Use tools if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [12]:
chat.run()

You: stop


Chat ended.


# Q3

In [13]:
#!pip install fastmcp

In [14]:
import fastmcp

In [15]:
fastmcp.__version__

'2.10.5'

# Q4

In [18]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [22]:
# weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

@mcp.tool
def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

if __name__ == "__main__":
    mcp.run()

/var/folders/9z/rhfc541d14q5klxp9lqvgx8m0000gq/T/ipykernel_30723/1625779597.py:41: RuntimeWarning: coroutine 'FastMCP.run_async' was never awaited
  mcp.run_async()


Response: **'stdio'**

# Q5

In [23]:
# {"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city":"berlin"}}}
# {"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city":"San Francisco"}}}

Response: **{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}**

# Q6

In [3]:
import asyncio
from fastmcp import Client

async def main():
    async with Client("mcp_server.py") as mcp_client:
        print(await mcp_client.list_tools())


if __name__ == "__main__":
    test = asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

# Additional

In [ ]:
import advanced_mcp_client as mcp_client
import chat_assistant

our_mcp_client = mcp_client.MCPClient(["python", "mcp_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

import json

import os
import dlt
import yaml
import requests
from openai import AzureOpenAI
from dataclasses import dataclass


@dataclass
class LLMServiceConfig:
    host: str
    api_key: str
    api_version: str
    model: str


with open("../OPENAI_API_KEY.yaml") as f:
    details = yaml.safe_load(f)["Glossary Terms Extraction Service"]

service_config = LLMServiceConfig(
    host=f"{details['protocol']}://{details['host']}",
    api_key=details['api_key'],
    api_version=details['api_version'],
    model=details['model']
)
client = AzureOpenAI(
    api_version=service_config.api_version,
    azure_endpoint=service_config.host,
    api_key=service_config.api_key
)
model = service_config.model


class MCPTools:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        self.tools = None

    def get_tools(self):
        if self.tools is None:
            mcp_tools = self.mcp_client.get_tools()
            self.tools = mcp_client.convert_tools_list(mcp_tools)
        return self.tools

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        result = self.mcp_client.call_tool(function_name, arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }

our_mcp_client = mcp_client.MCPClient(["python", "mcp_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()

Started server with command: python mcp_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.12.0'}}
Sending initialized notification...
Handshake completed successfully


/Users/vpankrat/Courses/LLMzoomcamp/venv/lib/python3.11/site-packages/pandas/core/groupby/generic.py:68: RuntimeWarning: coroutine 'main' was never awaited
  from pandas.core.frame import DataFrame


Started server with command: python mcp_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.12.0'}}
Sending initialized notification...
Handshake completed successfully


You: What's the weatjer in London? 


Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'London'}


...

Calling tool 'set_weather' with arguments: {'city': 'London', 'temp': 26.9}


You: What's the weather in Berlin? 


Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


You: What's the weather in London? 


Calling tool 'get_weather' with arguments: {'city': 'London'}
